# Tutorial 2

We use the same robot definition as in Tutorial 1.

In [ ]:
%matplotlib inline
import numpy as np

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; } </style>"))

In a similar way as in tutorial 1, we define a robot from an URDF-file.
Our new task specification deals with a laserspot shining on the ground plate.
The laser is following the z-axis at the end-effector.  The distance from the end
effector to the laserspot is modeled using a **feature variable** $d$, because we do not 
want to compute it explicitly.  In this case, it still possible to compute it explicitly,
but for more complicated surface, this will not the case anymore:

![Laser Skill](img/laser_skill.png)


We then state that the laserspot should be on the ground plane ( constraint `z` : coord_z(laserspot) == 0 == tgt_z),
and that it should follow the trajectory (constraints `x` and `z`).

To make things more interesting, we also impose some limits on the laser-distance (constraint `laserdistance`, distance should be between 0.45 [m] and 0.55 [m])

We generate output for the control errors and for the laserspot coordinates.

In [ ]:
from etasl_py.etasl import etasl_simulator,array_to_dict,dict_to_array,to_deg, to_rad
e = etasl_simulator(regularization_factor= 0.0000001)

e.readTaskSpecificationString("""
    require("context")
    require("geometric")
    local u=UrdfExpr();
    local fn = rospack_find("etasl_py_examples").."/robots/ur10_robot.urdf"
    u:readFromFile(fn)
    u:addTransform("ee","tool0","base_link")
    local r = u:getExpressions(ctx)
    ee = r.ee
    robot_jname={"shoulder_pan_joint","shoulder_lift_joint","elbow_joint","wrist_1_joint","wrist_2_joint","wrist_3_joint"}
    robot_jval = {}
    for i=1,#robot_jname do
       robot_jval[i]   = ctx:getScalarExpr(robot_jname[i])
    end
""")

e.readTaskSpecificationString(""" 

    tgt_x = ctx:createInputChannelScalar("tgt_x",0)
    tgt_y = ctx:createInputChannelScalar("tgt_y",0)
    tgt_z = ctx:createInputChannelScalar("tgt_z",0)


    d = Variable{context=ctx, name="d", vartype="feature"}


    Constraint{
        context=ctx,
        name="laserdistance",
        expr=d,
        target_lower = 0.3,
        target_upper = 0.45,
        K = 4
    }


    laserspot = ee*vector(0,0,d)

    Constraint{
        context=ctx,
        name="x",
        expr = tgt_x - coord_x(laserspot),
        priority = 2,
        K        = 4
    }
    Constraint{
        context=ctx,
        name="y",
        expr = tgt_y - coord_y(laserspot),
        priority = 2,
        K        = 4
    }
    Constraint{
        context=ctx,
        name="z",
        expr = tgt_z - coord_z(laserspot),
        priority = 2,
        K        = 4
    }

    ctx:setOutputExpression("error_x",coord_x(laserspot)-tgt_x)
    ctx:setOutputExpression("error_y",coord_y(laserspot)-tgt_y)
    ctx:setOutputExpression("error_z",coord_z(laserspot)-tgt_z)
    ctx:setOutputExpression("laser_x",coord_x(laserspot))
    ctx:setOutputExpression("laser_y",coord_y(laserspot))
    ctx:setOutputExpression("laser_z",coord_z(laserspot))
    
""")

In [ ]:
e.displayContext()

In this printed context, we can analyse the degrees of freedom:  we added one degree of freedom ( the feature variable d) to the 6 degrees of freedom of the robot and 3 constraints ( laserspot on the plane and following trajectory).  So, it the laser distance is not at its limits, we have 4 degree of freedom remaining. If the laser distance is at the limits, there are 3 degrees of freedom left.

We run the simulation in a similar way as in tutorial 1.  We choose a Lisajous figure as trajectory to follow.

In [ ]:

pos_lbl = ['shoulder_pan_joint','shoulder_lift_joint', 'elbow_joint', 
           'wrist_1_joint', 'wrist_2_joint',  'wrist_3_joint','d']
initial_jpos = np.array([0, -np.pi*0.6 , np.pi*0.6,-2.0,-np.pi*0.5,0.1,0.1])

N=4000
dt=0.005

time = np.arange(0,N)*dt
inp_lbl=['tgt_x','tgt_y','tgt_z']
inp=np.zeros((N, len(inp_lbl)))
f1=1
f2=2.5
inp[:,0] = np.sin(f1*time)*0.15 + 0.7
inp[:,1] = np.sin(f2*time)*0.1 +0.4
inp[:,2] = 0*time;
inpvel = np.zeros((N, len(inp_lbl)))
inpvel[:,0] = f1*np.cos(f1*time)*0.15
inpvel[:,1] = f2*np.cos(f2*time)*0.1
inpvel[:,2] = 0*time
e.setInputTable(inp_lbl,inp,inpvel)

e.initialize(initial_jpos, pos_lbl)
print(e.initial_jpos)
e.simulate(N=N,dt=dt)

# Plotting (using the Bokeh library to provide interactive plots)

In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import gridplot
from etasl_py.bokehplots import plotv, append_units

output_notebook()
print("positions : ")
plotv(e.TIME,e.POS, append_units(e.POS_LBL,[" [rad]"]*6 + [" [m]"]) )
print("velocities : ")
plotv(e.TIME,e.VEL, append_units(e.POS_LBL,[" [rad/s]"]*6 + [" [m/s]"]))
print("control errors : ")
plotv(e.TIME,e.OUTP[:,0:3],append_units(e.OUTP_LBL[0:3]," [m]"))

In [ ]:
f = figure(x_axis_label='x[m]', 
                y_axis_label= 'y[m]',plot_width=800, plot_height=400 )
f.line(e.INP[:,0],e.INP[:,1], line_width=2,color='red')
f.line(e.OUTP[:,3],e.OUTP[:,4], line_width=1)
show(f)

## Optimal propagation of the feedforward

As you can see at $time = 3s$ the task functions are completely converged, and there is no tracking error while tracking the known lisajous figure. We can test this to its extremes by setting the control constant to zero after $time=3s$. we can do this with a conditional expression:

    conditional(a,b,c)
      returns an expression that returns the following: 
      if a >=0 returns b, otherwise returns c 

We perform a simulation with the same input as before.
Again, we use Bokeh to generate plots.

In [ ]:
from etasl_py.etasl import etasl_simulator,array_to_dict,dict_to_array,to_deg, to_rad
e2 = etasl_simulator(regularization_factor= 0.0000001)

e2.readTaskSpecificationString("""
    require("context")
    require("geometric")
    local u=UrdfExpr();
    local fn = rospack_find("etasl_py_examples").."/robots/ur10_robot.urdf"
    u:readFromFile(fn)
    u:addTransform("ee","tool0","base_link")
    local r = u:getExpressions(ctx)
    ee = r.ee
    robot_jname={"shoulder_pan_joint","shoulder_lift_joint","elbow_joint","wrist_1_joint","wrist_2_joint","wrist_3_joint"}
    robot_jval = {}
    for i=1,#robot_jname do
       robot_jval[i]   = ctx:getScalarExpr(robot_jname[i])
    end
""")

e2.readTaskSpecificationString(""" 

    tgt_x = ctx:createInputChannelScalar("tgt_x",0)
    tgt_y = ctx:createInputChannelScalar("tgt_y",0)
    tgt_z = ctx:createInputChannelScalar("tgt_z",0)

    K = conditional(time-3, 0, 4)

    d = Variable{context=ctx, name="d", vartype="feature"}


    Constraint{
        context=ctx,
        name="laserdistance",
        expr=d,
        target_lower = 0.3,
        target_upper = 0.45,
        K = K
    }


    laserspot = ee*vector(0,0,d)

    Constraint{
        context=ctx,
        name="x",
        expr = tgt_x - coord_x(laserspot),
        priority = 2,
        K        = K
    }
    Constraint{
        context=ctx,
        name="y",
        expr = tgt_y - coord_y(laserspot),
        priority = 2,
        K        = K
    }
    Constraint{
        context=ctx,
        name="z",
        expr = tgt_z - coord_z(laserspot),
        priority = 2,
        K        = K
    }

    ctx:setOutputExpression("error_x",coord_x(laserspot)-tgt_x)
    ctx:setOutputExpression("error_y",coord_y(laserspot)-tgt_y)
    ctx:setOutputExpression("error_z",coord_z(laserspot)-tgt_z)
    ctx:setOutputExpression("K",K)
    ctx:setOutputExpression("laser_x",coord_x(laserspot))
    ctx:setOutputExpression("laser_y",coord_y(laserspot))
    ctx:setOutputExpression("laser_z",coord_z(laserspot))
    
""")

e2.setInputTable(inp_lbl,inp,inpvel)

e2.initialize(initial_jpos, pos_lbl)
print(e2.initial_jpos)
e2.simulate(N=N,dt=dt)

print("positions : ")
plotv(e2.TIME,e2.POS, append_units(e2.POS_LBL,[" [rad]"]*6 + [" [m]"]) )
print("velocities : ")
plotv(e2.TIME,e2.VEL, append_units(e2.POS_LBL,[" [rad/s]"]*6 + [" [m/s]"]))
print("control errors : ")
plotv(e2.TIME,e2.OUTP[:,0:4],append_units(e2.OUTP_LBL[0:4]," [m]"))

In [ ]:
#import matplotlib.pylab as plt
#print(e2.OUTP_LBL)
#plt.figure(figsize=(10,7))
#plt.plot(e2.INP[:,0], e2.INP[:,1],'r.',markersize=6)
#plt.plot(e2.OUTP[:,4],e2.OUTP[:,5],'.',markersize=2)
#plt.xlabel('x [m]')
#plt.ylabel('y [m]')



f = figure(x_axis_label='x[m]', 
                y_axis_label= 'y[m]',plot_width=800, plot_height=400 )
f.line(e2.INP[:,0],e2.INP[:,1], line_width=2,color='red')
f.line(e2.OUTP[:,4],e2.OUTP[:,5], line_width=1)
show(f)

Looking at the tracking error for x in detail, we see that the tracking error remains close to zero, except for
a **drift** caused by integration errors:

In [ ]:
f = figure(x_axis_label='time [s]', 
                y_axis_label= e2.OUTP_LBL[1],plot_width=800, plot_height=400, y_range=(-0.005,0.005)) 
f.line(e2.TIME, e2.OUTP[:,1], line_width=2)
show(f)
